In [1]:
import sys
sys.path.append("../")

In [2]:
import os

import pandas as pd
import joblib

from sklearn.ensemble import RandomForestRegressor

import os.path as op
import random

import matplotlib.pyplot as plt
import yaml

from agents.matchmanager import MatchManager
from agents.ml.regressionMulti import RegressionMultiAgent
from agents import GreedyAgent,RandomAgent
from agents.matchmanager import MatchManager
from agents.ml.classifier import ClassifierAgent
from agents.ml.regression import RegressionAgent
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

from core.const import RED, BLUE
from scenarios import scenarioJunction

from core.vectors import vectorAction, vectorActionInfo, vectorBoard, vectorBoardInfo
from core.game import GameBoard, GameState, vectorState, vectorStateInfo


import warnings

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,4)

In [3]:
with open(op.join('.', '../logger.config.yaml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)
# logging.config.dictConfig(config)

In [4]:
raw = pd.read_pickle('../data.2021-02-27.scenarioJunction.pkl.gz')\
        .drop(['meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'], axis=1, errors="ignore")

In [5]:
df = raw.dropna(axis=1, how='all')

In [6]:
df_red = df[df['action_team'] == 'red'].copy().drop('action_team', axis=1)
df_blue = df[df['action_team'] == 'blue'].copy().drop('action_team', axis=1)

In [8]:
df_red['label'] = df_red['winner'].apply(lambda x: 1 if x == 'red' else -1)
df_blue['label'] = df_blue['winner'].apply(lambda x: 1 if x == 'blue' else -1)

In [11]:
df_red_move=df_red.loc[((df_red['action_type_Move'] == True)|(df_red['action_type_MoveLoadInto'] == True))].copy()
df_red_attack=df_red.loc[((df_red['action_type_Attack'] == True)| (df_red['action_type_AttackGround'] == True) |(df_red['action_type_AttackRespond'] == True))].copy()
df_red_pass=df_red.loc[((df_red['action_type_Pass'] == True)| (df_red['action_type_PassFigure'] == True) |(df_red['action_type_PassTeam'] == True) | (df_red['action_type_PassRespond'] == True))].copy()

In [12]:
df_blue_move=df_blue.loc[((df_blue['action_type_Move'] == True)|(df_blue['action_type_MoveLoadInto'] == True))].copy()
df_blue_attack=df_blue.loc[((df_blue['action_type_Attack'] == True)| (df_blue['action_type_AttackGround'] == True) |(df_blue['action_type_AttackRespond'] == True))].copy()
df_blue_pass=df_blue.loc[((df_blue['action_type_Pass'] == True)| (df_blue['action_type_PassFigure'] == True) |(df_blue['action_type_PassTeam'] == True) | (df_blue['action_type_PassRespond'] == True))].copy()

In [13]:
X_red = df_red.drop(['winner', 'label'], axis=1)
y_red = df_red['label']

X_blue = df_blue.drop(['winner', 'label'], axis=1)
y_blue = df_blue['label']

In [14]:
X_red_move = df_red_move.drop(['winner', 'label'], axis=1)
y_red_move = df_red_move['label']

X_blue_move = df_blue_move.drop(['winner', 'label'], axis=1)
y_blue_move = df_blue_move['label']

In [15]:
X_red_attack = df_red_attack.drop(['winner', 'label'], axis=1)
y_red_attack  = df_red_attack['label']

X_blue_attack  = df_blue_attack.drop(['winner', 'label'], axis=1)
y_blue_attack  = df_blue_attack['label']

In [16]:
X_red_pass = df_red_pass.drop(['winner', 'label'], axis=1)
y_red_pass  = df_red_pass['label']

X_blue_pass  = df_blue_pass.drop(['winner', 'label'], axis=1)
y_blue_pass  = df_blue_pass['label']

In [17]:
X = pd.concat([X_red, X_blue])
y = pd.concat([y_red, y_blue])

In [26]:
rfr_red = RandomForestRegressor()
rfr_blue = RandomForestRegressor()
rfr = RandomForestRegressor()

rfr_red.fit(X_red, y_red)
rfr_blue.fit(X_blue, y_blue)
rfr.fit(X, y)

joblib.dump(rfr_red, '../models/Junction_RandomForestRegressor_red_20210227.joblib')
joblib.dump(rfr_blue, '../models/Junction_RandomForestRegressor_blue_20210227.joblib')
joblib.dump(rfr, '../models/Junction_RandomForestRegressor_20210227.joblib')

['../models/Junction_RandomForestRegressor_20210227.joblib']

In [27]:
rfc_red = RandomForestClassifier()
rfc_blue = RandomForestClassifier()
rfc = RandomForestClassifier()

rfc_red.fit(X_red, y_red)
rfc_blue.fit(X_blue, y_blue)
rfc.fit(X, y)

joblib.dump(rfc_red, '../models/Junction_RandomForestClassifier_red_20210227.joblib')
joblib.dump(rfc_blue, '../models/Junction_RandomForestClassifier_blue_20210227.joblib')
joblib.dump(rfc, '../models/Junction_RandomForestClassifier_20210227.joblib')

['../models/Junction_RandomForestClassifier_20210227.joblib']

In [28]:
rfr_red_move = RandomForestRegressor()
rfr_blue_move = RandomForestRegressor()
rfr_red_attack = RandomForestRegressor()
rfr_blue_attack = RandomForestRegressor()
rfr_red_pass = RandomForestRegressor()
rfr_blue_pass = RandomForestRegressor()
rfr = RandomForestRegressor()

rfr_red_move.fit(X_red_move, y_red_move)
rfr_red_attack.fit(X_red_attack, y_red_attack)
rfr_red_pass.fit(X_red_pass, y_red_pass)

rfr_blue_move.fit(X_blue_move, y_blue_move)
rfr_blue_attack.fit(X_blue_attack, y_blue_attack)
rfr_blue_pass.fit(X_blue_pass, y_blue_pass)


rfr.fit(X, y)

joblib.dump(rfr_red_move, '../models/Junction_RandomForestRegressor_red_move_20210227.joblib')
joblib.dump(rfr_red_attack, '../models/Junction_RandomForestRegressor_red_attack_20210227.joblib')
joblib.dump(rfr_red_pass, '../models/Junction_RandomForestRegressor_red_pass_20210227.joblib')


joblib.dump(rfr_blue_move, '../models/Junction_RandomForestRegressor_blue_move_20210227.joblib')
joblib.dump(rfr_blue_attack, '../models/Junction_RandomForestRegressor_blue_attack_20210227.joblib')
joblib.dump(rfr_blue_pass, '../models/Junction_RandomForestRegressor_blue_pass_20210227.joblib')



['../models/Junction_RandomForestRegressor_blue_pass_20210227.joblib']